<a href="https://colab.research.google.com/github/eisbetterthanpi/pytorch/blob/main/Unit_1_huggingface_rl_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install python-opengl ffmpeg xvfb
!pip3 install pyvirtualdisplay
# Virtual display: virtual screen to be able to render the environment (and thus record the frames)
from pyvirtualdisplay import Display
virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

!pip install gym[box2d] stable-baselines3[extra] huggingface_sb3 pyglet ale-py==0.7.4
import gym
# https://huggingface.co/models?pipeline_tag=reinforcement-learning&sort=downloads
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env


In [ ]:
# env = gym.make("LunarLander-v2")
env = make_vec_env('LunarLander-v2', n_envs=16)
model = PPO(
    policy = 'MlpPolicy', #MultiLayerPerceptron for when input is a vector
    # policy = 'CnnPolicy', #for when input are frames
    env = env,
    n_steps = 1024,
    batch_size = 64,
    n_epochs = 4,
    gamma = 0.999,
    gae_lambda = 0.98,
    ent_coef = 0.01,
    verbose=1)

# model.learn(total_timesteps=500000) # Train the agent
model.save("ppo-LunarLander-v2") # Save the model

Using cuda device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 93.2     |
|    ep_rew_mean     | -178     |
| time/              |          |
|    fps             | 2757     |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 16384    |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 85.2        |
|    ep_rew_mean          | -137        |
| time/                   |             |
|    fps                  | 1679        |
|    iterations           | 2           |
|    time_elapsed         | 19          |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.009773658 |
|    clip_fraction        | 0.0525      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.38       |
|    explained_variance   | 0.000372    |
|    learnin

KeyboardInterrupt: ignored

In [ ]:
# evaluate
eval_env = gym.make("LunarLander-v2")
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")
# https://stable-baselines3.readthedocs.io/en/master/guide/examples.html#basic-usage-training-saving-loading
# mean_reward=-21.81 +/- 17.647
############### end ###################